## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-09-08-33-00 +0000,wsj,Malaysia’s Central Bank Cuts Rates for First T...,https://www.wsj.com/articles/malaysias-central...
1,2025-07-09-08-32-47 +0000,bbc,More than 100 hurt as efforts to contain Marse...,https://www.bbc.com/news/articles/cp8mz44j6n6o
2,2025-07-09-08-26-29 +0000,nypost,Hundreds of earthquakes detected at Washington...,https://nypost.com/2025/07/09/us-news/hundreds...
3,2025-07-09-08-25-56 +0000,bbc,Anger after sharp rise in death toll from Keny...,https://www.bbc.com/news/articles/c9dgv5e6447o
4,2025-07-09-08-12-06 +0000,bbc,Musk's AI firm deletes posts after chatbot pra...,https://www.bbc.com/news/articles/c4g8r34nxeno


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2350/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
138,trump,10
170,tariffs,5
59,uk,5
67,new,4
94,will,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
19,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...,36
41,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,35
44,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...,28
33,2025-07-09-01-00-00 +0000,wsj,Two Republicans named Kevin are vying to be th...,https://www.wsj.com/economy/central-banking/tr...,27
17,2025-07-09-05-04-56 +0000,nypost,Trump insists ‘no amnesty’ for migrant farmwor...,https://nypost.com/2025/07/09/us-news/trump-in...,22


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
19,36,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...
33,27,2025-07-09-01-00-00 +0000,wsj,Two Republicans named Kevin are vying to be th...,https://www.wsj.com/economy/central-banking/tr...
18,21,2025-07-09-04-00-08 +0000,nypost,Long-lost bow of WWII ship found nearly 83 yea...,https://nypost.com/2025/07/09/us-news/long-los...
41,20,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
42,19,2025-07-08-21-31-55 +0000,bbc,King and Macron toast 'ever closer' UK-France ...,https://www.bbc.com/news/articles/cvg87y6d5j4o
23,17,2025-07-09-02-52-23 +0000,nypost,Rep. Ilhan Omar’s activist daughter unemployed...,https://nypost.com/2025/07/08/us-news/rep-ilha...
24,17,2025-07-09-02-34-52 +0000,nypost,New Mexico town suffers damage again as heavy ...,https://nypost.com/2025/07/08/us-news/new-mexi...
27,16,2025-07-09-01-52-10 +0000,nypost,Lincoln Tunnel closed for an hour as raging ca...,https://nypost.com/2025/07/08/us-news/lincoln-...
29,16,2025-07-09-01-49-09 +0000,nypost,College group Zohran Mamdani co-founded welcom...,https://nypost.com/2025/07/08/us-news/college-...
44,16,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
